In [76]:
import pandas as pd

In [77]:
# Pizzas
pizzas = pd.read_csv('ORIGINALS/pizzas.csv')
pizzas.head()

,pizza_id,pizza_type_id,size,price
0,bbq_ckn_s,bbq_ckn,S,12.75
1,bbq_ckn_m,bbq_ckn,M,16.75
2,bbq_ckn_l,bbq_ckn,L,20.75
3,cali_ckn_s,cali_ckn,S,12.75
4,cali_ckn_m,cali_ckn,M,16.75


In [78]:
# pizza_types
pizza_types = pd.read_csv('ORIGINALS/pizza_types.csv', encoding='latin-1')
pizza_types.head()

,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."


In [79]:
# orders
orders = pd.read_csv('ORIGINALS/orders.csv')
orders.head()

,order_id,date,time
0,1,01/01/2015,11:38:36
1,2,01/01/2015,11:57:40
2,3,01/01/2015,12:12:28
3,4,01/01/2015,12:16:31
4,5,01/01/2015,12:21:30


In [80]:
# order_details
order_details = pd.read_csv('ORIGINALS/order_details.csv')
order_details.head()

,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1


In [81]:
# data_dictionary
data_dictionary = pd.read_csv('ORIGINALS/data_dictionary.csv')
data_dictionary.head()

,Table,Field,Description
0,orders,order_id,Unique identifier for each order placed by a t...
1,orders,date,Date the order was placed (entered into the sy...
2,orders,time,Time the order was placed (entered into the sy...
3,order_details,order_details_id,Unique identifier for each pizza placed within...
4,order_details,order_id,Foreign key that ties the details in each orde...


In [1]:
def search_pizza(pizza_id):
    pizza = pizzas[pizzas['pizza_id'] == pizza_id]
    return pizza['pizza_type_id'].values[0], pizza['size'].values[0]


def create_csv_orders(orders, order_details):
    ordered_pizzas = [[] for i in range(len(orders))]
    for i in order_details['order_details_id']:
        order = order_details[order_details['order_details_id'] == i]
        ordered_pizzas[int(order['order_id'])-1].append(order['pizza_id'].values[0])
    orders['pizzas'] = ordered_pizzas
    orders.to_csv('TRANSFORMED/ordered_pizzas.csv', index=False)
    return orders


def count_pizzas(ordered_pizzas, pizza_types):
    lst = [0 for i in range(len(pizza_types))]
    weigths = {'S': 1, 'M': 1.5, 'L': 2, 'XL': 2.5, 'XXL': 3}
    pizza_counts = {'Monday': lst.copy(), 'Tuesday': lst.copy(), 'Wednesday': lst.copy(), 'Thursday': lst.copy(), 'Friday': lst.copy(), 'Saturday': lst.copy(), 'Sunday': lst.copy()}
    for _, order in ordered_pizzas.iterrows():
        day = order['day']
        pizzas = order['pizzas']
        for pizza in pizzas:
            pizza_flavour, size = search_pizza(pizza)
            ind = pizza_types[pizza_types['pizza_type_id'] == pizza_flavour].index.values[0]
            pizza_counts[day][ind] += 1*weigths[size]
    for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
        pizza_types[day] = pizza_counts[day]
    pizza_types.to_csv('TRANSFORMED/pizza_counts.csv', index=False)
    return pizza_types


def week_days(orders):
    dates = []
    for date in orders['date']:
        dates.append(date_to_day(date))
    orders['day'] = dates
    orders.to_csv('TRANSFORMED/ordered_pizzas.csv', index=False)
    return orders
    

def date_to_day(date):
    list_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    days_per_month = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
    date = date.split('/')
    days = 0
    for i in range(1, int(date[1])):
        days += days_per_month[i]
    days += int(date[0]) - 1
    return list_days[(days+3) % 7]


In [83]:
ordered_pizzas = create_csv_orders(orders, order_details)

In [ ]:
order_day = week_days(orders)

In [ ]:
pizza_counts = count_pizzas(ordered_pizzas, pizza_types)